## No Lambdas

Example: 1 in the second to last place

<img src = "../Week3_NFAs/1in2splace.png">

In [ ]:
def eval_nfa(s, q, delta, F, verbose=False):
    """
    Evaluate a standard NFA with no lambda arrows
    
    Parameters
    ----------
    s: string
        Input to try
    q: string
        Start state
    delta: (string, string) -> set([string])
        Transition function
    F: set([string])
        Accept states
    verbose: bool
        If True, print info about sequence of states that are visited
    """
    states = set([q]) ## We now track multiple possibilities
    if verbose:
        print("Start", states)
    for c in s:
        next_states = set([])
        for p in states:
            if (p, c) in delta:
                ## This line is different from DFA; there are multiple
                ## states we can branch out to; track the unique ones
                next_states = next_states.union(delta[(p, c)])
        states = next_states
        if verbose:
            print(c, states)
    ## At least one state must be in the set of accept states F
    return len(states.intersection(F)) > 0


delta = {
    ("q", "0"):{"q"},
    ("q", "1"):{"q", "q2"},
    ("q2", "0"):{"q0"},
    ("q2", "1"):{"q0"}
}

F = {"q0"}
q = "q"
s = "11010"
print(s, "Accepts:", eval_nfa(s, q, delta, F, verbose=True))

## With Lambdas


In [ ]:
def get_reachable_lambdas(start, delta):
    """
    Parameters
    ----------
    start: string
        State at which to start.  We'll gather anything
        that can be reached by a sequence of lambdas
        from this state
    delta: (string, string) -> set([string])
        Transition function
    
    Returns
    -------
    set([string])
        Set of all states reachable from start 
        by a sequence of lambda arrows
    """
    reachable = set([])
    stack = [start]
    while len(stack) > 0:
        state = stack.pop()
        reachable.add(state)
        if (state, None) in delta:
            # If there are lambda arrows out of this state, 
            # loop through all of them
            for neighbor in delta[(state, None)]:
                if not neighbor in reachable:
                    # If we haven't seen this state yet 
                    # (Crucial to avoid infinite loops for lambda cycles)
                    stack.append(neighbor)
    reachable.remove(start) # Exclude the start state itself
    return reachable
            
def reduce_nfa_lambdas(delta, F):
    """
    Eliminate the lambdas in an NFA by doing a reduction
    to equivalent arrows and accept states to without 
    lambdas
     
    Parameters
    ----------
    delta: (string, string) -> set([string])
        Transition function, which will be updated as as side effect
    F: set([string])
        Final states, which will be updated as as side effect
    """
    ## Step 1: Copy the dictionary into a new format where
    ## it's more convenient to look up only the arrows coming
    ## out of a particular state
    delta_state = {} #(State):(Character:set(states))
    for (state, c), states_to in delta.items():
        if not state in delta_state:
            delta_state[state] = {}
        if not c in delta_state[state]:
            delta_state[state][c] = set([])
        delta_state[state][c].update(states_to)

    ## Step 2: Loop through each state, get the new arrows, and 
    ## update the final states
    states = set([key[0] for key in delta])
    F_new = set([])
    for state in states:
        # Get states that are reachable from this state
        reachable = get_reachable_lambdas(state, delta)
        # Figure out if this should be an accept state based
        # on what's reachable
        if len(reachable.intersection(F)) > 0:
            F_new.add(state)
        # Add the new arrows
        for other in reachable:
            if other in delta_state: # If there are actually any arrows coming out of this state
                for c, states_to in delta_state[other].items():
                    if c is not None:
                        # Add this new equivalent arrow
                        if not (state, c) in delta:
                            delta[(state, c)] = set([])
                        delta[(state, c)] = delta[(state, c)].union(states_to)
    print("F_new", F_new)
    F.update(F_new)
     
    ## Step 3: Remove the original lambda arrows
    to_remove = [key for key in delta.keys() if key[1] is None]
    for key in to_remove:
        del delta[key]
    

delta = {
    ("q0", "0"):{"q0"},
    ("q0", None):{"q1"},
    
    ("q1", "0"):{"q1"},
    ("q1", "1"):{"q3"},
    ("q1", None):{"q2"},
    
    ("q2", "0"):{"q2"},
    ("q2", "1"):{"q1"},
    ("q2", None):{"q3"},
    
    ("q3", "0"):{"q3"},
    ("q3", None):{"q1"}
}
F = {"q2"}

for key, value in delta.items():
    print(key, ":", value)

print("\n\nReduced:")
reduce_nfa_lambdas(delta, F)
for key in sorted(delta.keys()):
    print(key, ":", sorted(delta[key]))
print("F", sorted(F))

### Union Example Again

In [ ]:
delta = {
    ("q", None):{"q0", "peven"},
     
    ("q0", "0"):{"q0"},
    ("q0", "1"):{"q1"},
    ("q1", "0"):{"q1"},
    ("q1", "1"):{"q2"},
    ("q2", "0"):{"q2"},
    ("q2", "1"):{"q2"},
     
    ("peven", "0"):{"podd"},
    ("peven", "1"):{"peven"},
    ("podd", "0"):{"peven"},
    ("podd", "1"):{"podd"}
}
q = "q"
F = {"q0", "q1", "podd"}
s = "00011"
 
reduce_nfa_lambdas(delta, F)
print(s, "Accepts:", eval_nfa(s, q, delta, F, verbose=True))

In [ ]:
delta = {
    ("q", None):{"q0", "peven"},
    
    ("q0", "0"):{"q0"},
    ("q0", "1"):{"q1"},
    ("q1", "0"):{"q1"},
    ("q1", "1"):{"q2"},
    ("q2", "0"):{"q2"},
    ("q2", "1"):{"q2"},
    
    ("peven", "0"):{"podd"},
    ("peven", "1"):{"peven"},
    ("podd", "0"):{"peven"},
    ("podd", "1"):{"podd"}
}
q = "q"
F = {"q0", "q1", "podd"}
s = "00011"
print(s, "Accepts:", eval_nfa_withlambda(s, q, delta, F, verbose=True))

## More Complicated Example with Lambdas


In [ ]:
delta = {
    ("q0", None):{"q1"},
    ("q0", "0"):{"q0"},
    ("q0", "1"):{"q1"},
     
    ("q1", "1"):{"q1"},
    ("q1", None):{"q2"},
     
    ("q2", "0"):{"q2"}
}
q = "q0"
F = {"q2"}
 
reduce_nfa_lambdas(delta, F)
print("0011", "Accepts:", eval_nfa(s, q, delta, F, verbose=True))


"""
for key in sorted(delta.keys()):
    print(key, ":", delta[key])
print("Final", F)


strings = ""

N = 100
stack = [""]
for i in range(N):
    s = stack.pop(0)
    strings += " " + s
    stack.append(s+"0")
    stack.append(s+"1")
    print(s, eval_nfa(s, q, delta, F))

print(strings)
"""